In [1]:
import pandas as pd
import datetime as dt
import math
import re
import string
import numpy as np
import csv
import fnmatch
import os
from nltk import ngrams

In [2]:
    def one_row_n_grams(wordlist, n, trim=True, vector_cell = True):
        # print('----------------------------------')
        # print('One row wordlist: ', wordlist, len(wordlist))
        if (vector_cell):
            # print('vector_cell: ', vector_cell, 'Wordlist:', wordlist)
            wordlist = ''.join(wordlist)
            # print('Wordlist: ', wordlist)
        try:
            if (wordlist[0] == 'b'):
                wordlist = wordlist[1:]
        except:
            # print('except')
            return ()
        if(trim):
            wordlist = wordlist[1:-1]
        # print('wordlist:', wordlist)
        # print('len:',len(wordlist.split()))
        if(len(wordlist.split()) < n):
            n = len(wordlist.split())
        t_list = []
        for i in range(len(wordlist.split())-(n-1)):
            temp = wordlist.lower().replace('"', '').replace("'", "").replace('!', '') \
                  .replace("?", "").replace(",", "").replace(".", "").replace("#", "") \
                       .replace("@", "").replace(":", "").replace(";", "").replace("/", "")\
                       .replace("-", "").replace("\\n", "").replace("\\","").replace("~","").split()[i:i+n]
            # temp = temp.replace("\\", "")
            # print('temp:', temp)
            if(len(temp)> 0):
                t_list.append(temp)
        # print('t_list:',t_list)
        return (t_list)


In [3]:
 def match_ngrams(vector, level, n, trim, vector_cell):
        # print('n:',n)
        result = []
        # for item in vector:
        for row in list(vector):
            # print('----------------------------------')
            found = '!'
            # print('row', row)
            try:
                # print('Try, row:',row)
                n_gram_row = one_row_n_grams(row, n, trim,vector_cell)
                # print(n_gram_row)
                # print('n_gram',n_gram_row)
                for each in n_gram_row:
                    # used when top lists are from file
                    # print('cell:',each[0], len(each[0]),'level:',level[0], len(level[0]))
                    if (each[0] == level[0]):
                        # print('true')
                        found = level[0]
                    # used with top list is being discovered
                    # print('cell:',each[0],'level:',level)
                    # if (each[0] == level):
                    #     print('true')
                    #     found = level
            except:
                found = '!'
            result.append(found)
        # print('result', result)
        return (result)

In [4]:
    def add_new_col_df(col_prefix , list,column_name, df, n, trim_text, word_vector):
        i = 1
        l = len(list)
        for level in list:
            # print('i', i, 'len', l)
            # print(level)
            # print(level[0])
            if(i % 250 == 0 ):
                print('Added ',i, 'out of ', l, 'columns. - ',dt.datetime.now())
            new_col = col_prefix + str(level[0])
            df[new_col] = match_ngrams(df[column_name], level, n, trim_text, word_vector)
            i += 1

In [5]:
print(dt.datetime.now())
# path is where raw files are 
path = 'C:/Users/peted/Documents/Python/Raw_Files/'
# path_2 is where to put files after processing and where
# variable files are kept: top_desc_list_v6.csv, top_hashtags_v5.csv, top_word_list_v6.csv
path_2 = 'C:/Users/peted/Documents/Python/Raw_Files/'

2018-03-25 22:23:37.561408


In [7]:
objects = fnmatch.filter(os.listdir(path), '*_step_*')
i = 1
for load_file in objects:
    # to load goldset
    # load_file = 'Training_Gun_Tweets_v6.csv'
    print(path,load_file)

    df_tweet = pd.read_csv(path + load_file, encoding='utf-8')

    # for testing
    df_tweet = df_tweet[:10]

    col_count = 0

    print('hashtag -----------------------------------------------------')
    n = 1
    trim_text = True
    word_vector = False
    # top list
    file = 'top_hashtags_v5.csv'
    with open(path_2 + file, 'r', encoding='utf-8') as f:
      reader = csv.reader(f)
      top_hashtags = list(reader)
    print('top:',len(top_hashtags))
    # print('top:','len:', len(top_hashtags),top_hashtags)
    print('------------------------------------------------------')
    # create new columns (Random Forest can only take 32 levels for any given column
    add_new_col_df('#_',top_hashtags, 'hashtags', df_tweet, n, trim_text, word_vector)
    print('done')

    # get list of words
    print('words -----------------------------------------------------')
    # ngrams
    n = 1
    min_count = 0.5
    trim_text = True
    word_vector = False
    # # top list
    file = 'top_word_list_v6.csv'
    with open(path_2 + file, 'r', encoding='utf-8') as f:
      reader = csv.reader(f)
      top_list = list(reader)
    print('top:', len(top_list))
    # print('top:','len:', len(top_list),top_list)
    print('------------------------------------------------------')
    # create new columns (Random Forest can only take 32 levels for any given column
    add_new_col_df('t_', top_list, 'text', df_tweet, n, trim_text, word_vector)
    print('done')

    first_col = df_tweet.shape[1]+1
    # # # get list of words in user desc
    print('user desc -----------------------------------------------------')
    # ngrams
    n = 1
    min_count = 0.5
    trim_text = False
    word_vector = False
    # top list
    file = 'top_desc_list_v6.csv'
    with open(path_2 + file, 'r', encoding='utf-8') as f:
      reader = csv.reader(f)
      top_desc = list(reader)
    print('top:', len(top_desc))
    # print('top:','len:', len(top_desc),top_desc)
    print('------------------------------------------------------')
    # create new columns (Random Forest can only take 32 levels for any given column
    add_new_col_df('u_',top_desc, 'user_desc', df_tweet, n, trim_text, word_vector)

    print('Save file')
    load = load_file[:-4]+'_ready_for_RF_2.csv'
    df_tweet.to_csv(path_2 + load, sep=',', na_rep='', index=False, encoding='utf-8')

    print(dt.datetime.now())
    print('done with file ',load_file, i)
    i += 1
    # break
print('done with all cycles')

C:/Users/peted/Documents/Python/Raw_Files/ guncontrol_2018-02-24_tweets_step_1.csv
hashtag -----------------------------------------------------
top: 399
------------------------------------------------------
Added  250 out of  399 columns. -  2018-03-25 22:26:47.057671
done
words -----------------------------------------------------
top: 1711
------------------------------------------------------
Added  250 out of  1711 columns. -  2018-03-25 22:26:47.547440
Added  500 out of  1711 columns. -  2018-03-25 22:26:48.016962
Added  750 out of  1711 columns. -  2018-03-25 22:26:48.600368
Added  1000 out of  1711 columns. -  2018-03-25 22:26:49.170325
Added  1250 out of  1711 columns. -  2018-03-25 22:26:49.781461
Added  1500 out of  1711 columns. -  2018-03-25 22:26:50.513493
done
user desc -----------------------------------------------------
top: 1420
------------------------------------------------------
Added  250 out of  1420 columns. -  2018-03-25 22:26:51.653955
Added  500 out of  14